# Loading Data

## Imports

In [1]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Flatten, LeakyReLU, ReLU, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, concatenate, Activation
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras.backend import clear_session
# from tensorflow.keras.optimizers import Adam, RMSprop , SGD
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# from tensorflow.keras.regularizers import L2
# from tensorflow.keras import metrics
from tensorflow.keras import layers
from tensorflow import keras

import tensorflow as tf

from os.path import join
from os import listdir
from shutil import copy

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Creating data loaders

In [2]:
train_df = pd.read_csv("df//train_df.csv")
val_df = pd.read_csv("df//val_df.csv")
train_df.head()

,path,pH
0,close_seg\6\6K--22-_jpg.rf.663142c881758efb4de...,6
1,close_seg\6\6k--75-_jpg.rf.42aa04547265d5890c7...,6
2,close_seg\6\6F--23-_jpg.rf.338da0e01de039e7b73...,6
3,close_seg\6\6F--44-_jpg.rf.1521f3dad55c9c68fd2...,6
4,close_seg\6\6k--13-_jpg.rf.47db8c03884b63f6b6f...,6


In [3]:
# print(len(listdir("jpgData//6")))
# print(len(listdir("jpgData//7")))
# print(len(listdir("jpgData//8")))
# print(len(listdir("jpgData//9")))

In [4]:
x_col = "path"
y_col = "pH"
batch_size = 32
epochs = 1024
lr = 1e-5
image_size = (32,32)
channels = 3
shuffle = True
class_mode ="raw"
color_mode = "rgb"

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    #  width_shift_range = 0.1,
                                    #  height_shift_range = 0.1, 
                                    #  zoom_range = 0.2,
                                    #  shear_range = 0.5,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     channel_shift_range = 64.0,
                                    #  brightness_range = (0.3,1.0),
                                    #  rotation_range = 15,
                                     )
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                              x_col=x_col, y_col=y_col, has_ext=True, 
                                              class_mode=class_mode, target_size=image_size, 
                                              batch_size=batch_size, color_mode = color_mode,
                                              shuffle = shuffle)
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(dataframe=val_df,
                                              x_col=x_col, y_col=y_col, has_ext=True, 
                                              class_mode=class_mode, target_size=image_size, 
                                              batch_size=batch_size,  color_mode = color_mode,
                                              shuffle = shuffle)


Found 512 validated image filenames.
Found 131 validated image filenames.


# Design Alternatives

## Design A: Nested U-Net

In [6]:
# Defining the Convolutional Block
def conv_block(inputs, num_filters):
	# Applying the sequence of Convolutional, Batch Normalization
	# and Activation Layers to the input tensor
	x = Sequential([
		# Convolutional Layer
		Conv2D(num_filters, 1, padding='same'),
		# Batch Normalization Layer
		BatchNormalization(),
		# Activation Layer
		ReLU(),
		# Convolutional Layer
		Conv2D(num_filters, 1, padding='same'),
		# Batch Normalization Layer
		BatchNormalization(),
		# Activation Layer
		ReLU()
	])(inputs)

	# Returning the output of the Convolutional Block
	return x

# Defining the Unet++ Model
def unet_plus_plus_model(input_shape=(image_size[0], image_size[1], channels), num_classes=1, deep_supervision=True):
	inputs = Input(shape=input_shape)

	# Encoding Path
	x_00 = conv_block(inputs, 64)
	x_10 = conv_block(MaxPooling2D()(x_00), 128)
	x_20 = conv_block(MaxPooling2D()(x_10), 256)
	x_30 = conv_block(MaxPooling2D()(x_20), 512)
	x_40 = conv_block(MaxPooling2D()(x_30), 1024)

	# Nested Decoding Path
	x_01 = conv_block(concatenate(
		[x_00, UpSampling2D()(x_10)]), 64)
	x_11 = conv_block(concatenate(
		[x_10, UpSampling2D()(x_20)]), 128)
	x_21 = conv_block(concatenate(
		[x_20, UpSampling2D()(x_30)]), 256)
	x_31 = conv_block(concatenate(
		[x_30, UpSampling2D()(x_40)]), 512)

	x_02 = conv_block(concatenate(
		[x_00, x_01, UpSampling2D()(x_11)]), 64)
	x_12 = conv_block(concatenate(
		[x_10, x_11, UpSampling2D()(x_21)]), 128)
	x_22 = conv_block(concatenate(
		[x_20, x_21, UpSampling2D()(x_31)]), 256)

	x_03 = conv_block(concatenate(
		[x_00, x_01, x_02, UpSampling2D()(x_12)]), 64)
	x_13 = conv_block(concatenate(
		[x_10, x_11, x_12, UpSampling2D()(x_22)]), 128)

	x_04 = conv_block(concatenate(
		[x_00, x_01, x_02, x_03, UpSampling2D()(x_13)]), 64)

	# Deep Supervision Path
	# If deep supervision is enabled, then the model will output the segmentation maps
	# at each stage of the decoding path
	if deep_supervision:
		outputs = [
			Conv2D(num_classes, 1)(x_01),
			Conv2D(num_classes, 1)(x_02),
			Conv2D(num_classes, 1)(x_03),
			Conv2D(num_classes, 1)(x_04)
		]
		# Concatenating the segmentation maps
		outputs = concatenate(outputs, axis=0)

	# If deep supervision is disabled, then the model will output the final segmentation map
	# which is the segmentation map at the end of the decoding path
	else:
		flatten  = layers.Flatten()(x_04)
		dense = Dense(1024, activation='relu')(flatten)
		# dense = Dense(512, activation='relu')(dense)
		outputs = Dense(1)(dense)

	# Creating the model
	model = tf.keras.Model(
		inputs=inputs, outputs=outputs, name='Unet_plus_plus')

	# Returning the model
	return model




In [7]:
# Creating the model
modelA = unet_plus_plus_model(input_shape=(
	image_size[0], image_size[1], channels), deep_supervision=False)
modelA.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='mean_squared_error', metrics=['mae'])
modelA.summary()

Model: "Unet_plus_plus"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32, 32, 64)   4928        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 16, 64)   0           ['sequential[0][0]']             
                                                                                                  
 sequential_1 (Sequential)      (None, 16, 16, 128)  25856       ['max_pooling2d[0][0]']          
                                                                                     

In [8]:


# historyA = modelA.fit(train_generator, epochs=10, validation_data=val_generator)


In [9]:
# ## Graphing Result of training
# plt.figure(figsize=(15,4))

# plt.subplot(121)
# plt.plot(historyA.history['mae'], color ='r')
# plt.plot(historyA.history['val_mae'])
# plt.title('mae')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mae')

# plt.subplot(122)
# plt.plot(historyA.history['loss'], color ='r')
# plt.plot(historyA.history['val_loss'])
# plt.title('loss')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mean squared error')

# # displaying the figure
# plt.show()

In [10]:
# ## Best model saved
# train_loss = historyA.history['loss']
# train_mae = historyA.history['mae']
# val_loss = historyA.history['val_loss']
# val_mae = historyA.history['val_mae']

# best_epoch = np.argmin(np.array(val_loss))

# print("Best Epoch: ", best_epoch)
# print("Final Train Loss: {:.4f}".format(train_loss[best_epoch]))
# print("Final Validation Loss: {:.4f}".format(val_loss[best_epoch]))
# print("Final Train MAE: {:.2f}".format(train_mae[best_epoch]*100))
# print("Final Test MAE: {:.2f}".format(val_mae[best_epoch]))

## Design B: SegNet

In [11]:
def segnet(input_shape):

    # Encoding layer
    img_input = Input(shape= input_shape)
    x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(img_input)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
    x = BatchNormalization(name='bn3')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
    x = BatchNormalization(name='bn5')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
    x = BatchNormalization(name='bn6')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
    x = BatchNormalization(name='bn7')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
    x = BatchNormalization(name='bn8')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
    x = BatchNormalization(name='bn9')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
    x = BatchNormalization(name='bn10')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
    x = BatchNormalization(name='bn11')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
    x = BatchNormalization(name='bn12')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
    x = BatchNormalization(name='bn13')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Dense(1024, activation = 'relu', name='fc1')(x)
    x = Dense(1024, activation = 'relu', name='fc2')(x)
    # Decoding Layer 
    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
    x = BatchNormalization(name='bn14')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
    x = BatchNormalization(name='bn15')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
    x = BatchNormalization(name='bn16')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
    x = BatchNormalization(name='bn17')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
    x = BatchNormalization(name='bn18')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
    x = BatchNormalization(name='bn19')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
    x = BatchNormalization(name='bn20')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
    x = BatchNormalization(name='bn21')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
    x = BatchNormalization(name='bn22')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
    x = BatchNormalization(name='bn23')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
    x = BatchNormalization(name='bn24')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
    x = BatchNormalization(name='bn25')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
    x = BatchNormalization(name='bn26')(x)
    x = Activation('sigmoid')(x)
    # pred = Reshape((192,256))(x)
    x = Flatten()(x)
    x=Dense(1024, activation='relu')(x)
    otuput=Dense(1, activation='linear')(x)
    
    model = Model(inputs=img_input, outputs=otuput)
    
    model.compile(optimizer= tf.keras.optimizers.Adam(lr=lr), loss= ["mse"]
                  , metrics=['mae'])
    return model

In [12]:

modelB = segnet(input_shape=image_size+(3,))
modelB.summary()


# model.save(savename)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 32, 32, 64)        1792      
                                                                 
 bn1 (BatchNormalization)    (None, 32, 32, 64)        256       
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 conv2 (Conv2D)              (None, 32, 32, 64)        36928     
                                                                 
 bn2 (BatchNormalization)    (None, 32, 32, 64)        256       
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0     

c:\Users\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
# hist = modelB.fit(train_generator, epochs= 10, validation_data= val_generator, verbose=1)

In [14]:
# modelB = segnet(input_shape=image_size+(3,))
# modelB.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='mse', metrics=['mae'])
# modelB.summary()

In [15]:
# ## MANUFACTURABILITY: TRAINING TIME
# import time 
# start = time.time()
# historyB = modelB.fit(train_generator, epochs=10, validation_data=val_generator, verbose=0)
# stop = time.time()
# print(f"Training time: {stop - start}s")



In [16]:
# ## Graphing Result of training
# plt.figure(figsize=(15,4))

# plt.subplot(121)
# plt.plot(historyB.history['mae'], color ='r')
# plt.plot(historyB.history['val_mae'])
# plt.title('mae')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mae')

# plt.subplot(122)
# plt.plot(historyB.history['loss'], color ='r')
# plt.plot(historyB.history['val_loss'])
# plt.title('loss')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mean squared error')

# # displaying the figure
# plt.show()

In [17]:
# ## Best model saved
# train_loss = historyB.history['loss']
# train_mae = historyB.history['mae']
# val_loss = historyB.history['val_loss']
# val_mae = historyB.history['val_mae']

# best_epoch = np.argmin(np.array(val_loss))

# print("Best Epoch: ", best_epoch)
# print("Final Train Loss: {:.4f}".format(train_loss[best_epoch]))
# print("Final Validation Loss: {:.4f}".format(val_loss[best_epoch]))
# print("Final Train MAE: {:.2f}".format(train_mae[best_epoch]*100))
# print("Final Test MAE: {:.2f}".format(val_mae[best_epoch]))

In [18]:
train_generator[1][1]

array([8, 8, 7, 9, 8, 9, 9, 6, 9, 6, 7, 9, 8, 7, 8, 6, 7, 6, 9, 9, 7, 8,
       6, 7, 8, 6, 9, 9, 7, 7, 6, 6], dtype=int64)

## Design C: DeepLabv3+


In [19]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def DeeplabV3Plus(image_size):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights=None, include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    x = layers.GlobalAveragePooling2D()(x)
    x= Dense(1024, activation="relu",)(x)
    model_output = layers.Dense(1, activation='linear')(x)
    return keras.Model(inputs=model_input, outputs=model_output)


modelC = DeeplabV3Plus(image_size=image_size[0])
modelC.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [20]:
modelC.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='mse', metrics=['mae'])

# historyC = modelC.fit(train_generator, epochs=10, validation_data=val_generator)


In [21]:
# ## Graphing Result of training
# plt.figure(figsize=(15,4))

# plt.subplot(121)
# plt.plot(historyC.history['mae'], color ='r')
# plt.plot(historyC.history['val_mae'])
# plt.title('mae')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mae')

# plt.subplot(122)
# plt.plot(historyC.history['loss'], color ='r')
# plt.plot(historyC.history['val_loss'])
# plt.title('loss')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mean squared error')

# # displaying the figure
# plt.show()

In [22]:
# ## Best model saved
# train_loss = historyC.history['loss']
# train_mae = historyC.history['mae']
# val_loss = historyC.history['val_loss']
# val_mae = historyC.history['val_mae']

# best_epoch = np.argmin(np.array(val_loss))

# print("Best Epoch: ", best_epoch)
# print("Final Train Loss: {:.4f}".format(train_loss[best_epoch]))
# print("Final Validation Loss: {:.4f}".format(val_loss[best_epoch]))
# print("Final Train MAE: {:.2f}".format(train_mae[best_epoch]*100))
# print("Final Test MAE: {:.2f}".format(val_mae[best_epoch]))

# Evaluation of Constraints

In [46]:
## Prepare set of x values and y values for performance constraint
X_values,y_values = next(val_generator)

## MANUFACTURABILITY: TRAINING TIME

In [24]:
## DESIGN A
import time 
start = time.time()
historyA = modelA.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=0)
stop = time.time()
print(f"Training time: {stop - start}s")

# ## FUNCTIONALITY: INFERENCE TIME
# modelC.evaluate(train_generator[1][0][0].reshape(1,32,32,3))

Training time: 800.670812368393s


In [25]:
## DESIGN B
import time 
start = time.time()
historyB = modelB.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=0)
stop = time.time()
print(f"Training time: {stop - start}s")

Training time: 935.2625951766968s


In [26]:
## DESIGN C
import time 
start = time.time()
historyC = modelC.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=0)
stop = time.time()
print(f"Training time: {stop - start}s")

Training time: 708.7862117290497s


## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND

In [27]:
## Design A
## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND
# Calculae FLOPS
from keras_flops import get_flops
flopsA = get_flops(modelA, batch_size=1)
print(f"FLOPS: {flopsA / 10 ** 9:.03} G")

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
FLOPS: 0.616 G


In [28]:
## Design B
## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND
# Calculae FLOPS
from keras_flops import get_flops
flopsB = get_flops(modelB, batch_size=1)
print(f"FLOPS: {flopsB / 10 ** 9:.03} G")

FLOPS: 1.28 G


In [29]:
## Design C
## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND
# Calculae FLOPS
from keras_flops import get_flops
flopsC = get_flops(modelC, batch_size=1)
print(f"FLOPS: {flopsC / 10 ** 9:.03} G")

FLOPS: 0.89 G


## FUNCTIONALITY: INFERENCE TIME

In [30]:
## Design A
## FUNCTIONALITY: INFERENCE TIME
resultA = modelA.evaluate(train_generator[1][0], return_dict=True)

1/1 [==============================] - 0s 357ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


In [31]:
## Design B
## FUNCTIONALITY: INFERENCE TIME
resultB = modelB.evaluate(train_generator[1][0], return_dict=True)

1/1 [==============================] - 0s 359ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


In [32]:
## Design AC
## FUNCTIONALITY: INFERENCE TIME
resultC = modelC.evaluate(train_generator[1][0], return_dict=True)

1/1 [==============================] - 1s 523ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


## PERFORMANCE: COEFFICIENT OF DETERMINATION

In [47]:
## Design A
## PERFORMANCE: COEFFICIENT OF DETERMINATION
from sklearn.metrics import r2_score
predictionsA = modelA.predict(X_values)
codA = r2_score(y_values,predictionsA)
print(codA)

1/1 [==============================] - 0s 20ms/step
0.7174338841123394


In [48]:
## Design B
## PERFORMANCE: COEFFICIENT OF DETERMINATION
from sklearn.metrics import r2_score
predictionsB = modelB.predict(X_values)
codB = r2_score(y_values,predictionsB)
print(codB)

1/1 [==============================] - 0s 23ms/step
0.6815232240210634


In [49]:
## Design C
## PERFORMANCE: COEFFICIENT OF DETERMINATION
from sklearn.metrics import r2_score
predictionsC = modelC.predict(X_values)
codC = r2_score(y_values,predictionsC)
print(codC)

1/1 [==============================] - 0s 20ms/step
0.34718149864239733


## EFFICIENCY: STORAGE CONSUMPTION

In [43]:
## EFFICIENCY: STORAGE CONSUMPTION
weightsA = modelA.get_weights()
total_sizeA = 0
for weight in weightsA:
    total_sizeA += tf.size(weight).numpy()

print(f"Total model weight size in megabytes: {total_sizeA*8e-6}")

Total model weight size in megabytes: 569.719304


In [44]:
## EFFICIENCY: STORAGE CONSUMPTION
weightsB = modelB.get_weights()
total_sizeB = 0
for weight in weightsB:
    total_sizeB += tf.size(weight).numpy()

print(f"Total model weight size in megabytes: {total_sizeB*8e-6}")

Total model weight size in megabytes: 275.554352


In [45]:
## EFFICIENCY: STORAGE CONSUMPTION
weightsC = modelC.get_weights()
total_sizeC = 0
for weight in weightsC:
    total_sizeC += tf.size(weight).numpy()

print(f"Total model weight size in megabytes: {total_sizeC*8e-6}")

Total model weight size in megabytes: 96.930312
